# Inisiasi Librarry and Variabel

In [32]:
import pandas as pd 
import numpy as np
import openpyxl
import subprocess
import requests
import os
import time
import re
from datetime import datetime
from openpyxl import Workbook
from tqdm import tqdm, trange
from bs4 import BeautifulSoup
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [33]:
tahun = input("Masukkan tahun (..., 2023, 2024, ...): ")
kuartal = input("Masukkan Kuartal (Q1, Q2, Q3, atau Audit): ")

data_xbrl_path = "./cons/datas/"
data_transit_path = "./cons/transit/"
data_hasil_path = "./hasil/"
folderpath = f"{data_xbrl_path}{tahun}_{kuartal}"

# XBRL Downloader From IDX

In [34]:
def log_error(message):
    with open("download_errors.log", "a") as log_file:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_file.write(f"[{timestamp}] {message}\n")

#Mencatat pesan kesalahan ke dalam file log.

In [35]:
def get_years_from_input():
    while True:
        user_input = input("Masukkan Tahun : ")
        try:
            tahun_list = [int(t.strip()) for t in user_input.split(",") if t.strip().isdigit()]
            if not tahun_list:
                raise ValueError("Tahun Tidak Valid !!!")
            return tahun_list
        except ValueError as e:
            print(f"Input tidak valid: {e}. Coba lagi.")

#Meminta input tahun dari pengguna dan memprosesnya menjadi daftar tahun yang valid.

In [ ]:
def get_kuartals_from_input():
    kuartal_dict = {
        "Q1": "I",
        "Q2": "II",
        "Q3": "III",
        "Audit": "Tahunan"
    }
    while True:
        user_input = input("Masukkan kuartal (Q1, Q2, Q3, Audit): ")
        kuartal_list = [k.strip() for k in user_input.split(",") if k.strip() in kuartal_dict]

        if not kuartal_list:
            print("Inputan Kuartal Tidak Valid !!!")
            continue

        return {k: kuartal_dict[k] for k in kuartal_list}

#Meminta input kuartal dari pengguna dan memprosesnya menjadi daftar kuartal yang valid.

In [ ]:
def read_company_codes(excel_file="daftar_files.xlsx", sheet_name="Tidak Terunduh"):
    try:
        df = pd.read_excel(excel_file, sheet_name=sheet_name)
        return df['Kode Perusahaan'].dropna().tolist()
    except Exception as e:
        log_error(f"Error membaca file Excel: {e}")
        print(f"Error membaca file Excel: {e}")
        return []
    
# Membaca daftar kode perusahaan dari file Excel.
# Kalau mau ganti nama file Daftar Perusahaan di sini

In [ ]:
# INI MAIN PROGRAMNYA

def download_files():

    tahun_list = get_years_from_input() # Inputan Tahun
    kuartal_dict = get_kuartals_from_input() # Inputan Kuartal
    kode_perusahaan_list = read_company_codes() # Membaca kode perusahaan dari file Excel

    if not kode_perusahaan_list:
        return

    # Iterasi tahun, kuartal, dan kode perusahaan
    for tahun in tahun_list:

        for kuartal, kuartal_romawi in kuartal_dict.items():

            for kode_perusahaan in kode_perusahaan_list:
                # Membuat direktori berdasarkan tahun dan kuartal
                download_dir = f"datas/{tahun}_{kuartal}"
                os.makedirs(download_dir, exist_ok=True)

                # Path lengkap file output
                output_file = os.path.join(download_dir, f"FinancialStatement-{tahun}-{kuartal_romawi}-{kode_perusahaan}.xlsx")
                
                # URL file yang akan diunduh
                url = (
                    f"https://idx.co.id/Portals/0/StaticData/ListedCompanies/Corporate_Actions/New_Info_JSX/Jenis_Informasi/01_Laporan_Keuangan/"
                    f"02_Soft_Copy_Laporan_Keuangan/Laporan%20Keuangan%20Tahun%20{tahun}/{kuartal}/{kode_perusahaan}/"
                    f"FinancialStatement-{tahun}-{kuartal_romawi}-{kode_perusahaan}.xlsx"
                )
                
                # Perintah curl
                command = [
                    "curl",
                    "-H", "User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/110.0",
                    url,
                    "-o", output_file
                ]
                
                # Eksekusi perintah
                try:
                    time.sleep(2)
                    subprocess.run(command, check=True)
                    print(f"File berhasil diunduh: {kode_perusahaan} {kuartal} {tahun}")
                    

                except subprocess.CalledProcessError as e:
                    error_message = f"Gagal mengunduh file: {kode_perusahaan}. Error: {e}"
                    log_error(error_message)
                    print(error_message)
                    time.sleep(4)


In [ ]:
if __name__ == "__main__":
    download_files()

# Validate File

In [ ]:
def write_codes_to_sheet(sheet, codes):

    sheet.cell(row=1, column=1).value = "No"
    sheet.cell(row=1, column=2).value = "Kode Perusahaan"

    for i, code in enumerate(codes):
        sheet.cell(row=i+2, column=1).value = i+1
        sheet.cell(row=i+2, column=2).value = code


In [ ]:
def process_files(folder_path):

    less_than_10kb = []
    greater_than_10kb = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(folder_path, filename)
            file_size = os.path.getsize(file_path)
            match = re.search(r"-([A-Z]+)\.xlsx$", filename)
            if match:
                code = match.group(1)
                if file_size < 10240:
                    less_than_10kb.append(code)
                else:
                    greater_than_10kb.append(code)
    return less_than_10kb, greater_than_10kb

In [ ]:
def write_to_excel(data_less, data_greater, output_file):

    workbook = Workbook()

    # Sheet 1: Less than 10KB
    sheet_less = workbook.active
    sheet_less.title = "Tidak Terunduh"
    write_codes_to_sheet(sheet_less, data_less)

    # Sheet 2: Greater than or equal to 10KB
    sheet_greater = workbook.create_sheet("Terunduh")
    write_codes_to_sheet(sheet_greater, data_greater)

    workbook.save(output_file)

In [ ]:
def hapus_sampah(folder_path, batas_kb=10):

    if not os.path.exists(folder_path):
        print(f"Folder '{folder_path}' tidak ditemukan.")
        return
    
    # Konversi batas KB ke byte
    size_limit = batas_kb * 1024  
    jumlah_dihapus = 0

    # Loop melalui semua file dalam folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Periksa apakah itu file (bukan folder)
        if os.path.isfile(file_path):
            file_size = os.path.getsize(file_path)  # Dapatkan ukuran file
            
            # Jika ukuran file kurang dari batas, hapus
            if file_size < size_limit:
                os.remove(file_path)
                jumlah_dihapus += 1
                print(f"File '{filename}' ({file_size} bytes) telah dihapus.")

    print(f"Proses selesai. {jumlah_dihapus} file telah dihapus.")

In [ ]:
if __name__ == "__main__":
    folder_path = input("Masukkan path folder: ")
    less_than_10kb, greater_than_10kb = process_files(folder_path)

    output_file = f"daftar_files.xlsx"
    write_to_excel(less_than_10kb, greater_than_10kb, output_file)

    hapus_sampah(folder_path)

    print(f"Kode perusahaan telah disimpan di {output_file}")

# Datas Scrapper

In [37]:
def file_info_scraper(folder_path):
    xlsx_files = [f for f in os.listdir(folder_path) if f.endswith(".xlsx")]
    data = {"NamaFile":[], "kode entitas": [], "NamaSheetPK": [], "NamaSheetLR": [], "NamaSheetRB": []}
        
    print("Mengumpulkan Nama Sheet...")
    for namefile in tqdm(xlsx_files):
        file_path = os.path.join(folder_path, namefile)
        try:
            xls = pd.ExcelFile(file_path)
        except Exception as e :
            print("\r",namefile, e)
            continue
            
        data["kode entitas"].append(namefile[-9:-5])
        sheet_pk, sheet_lr, sheet_rb = None, None, None
            
        for sheet in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet, nrows=10, dtype=str)
            text = " ".join(df.astype(str).fillna("").values.flatten())
                
            if "Statement of financial position" in text:
                sheet_pk = sheet
            if "Statement of profit or loss and other comprehensive income" in text:
                sheet_lr = sheet
            if "Statement of cash flows" in text:
                sheet_rb = sheet
                
        data["NamaSheetPK"].append(sheet_pk)
        data["NamaSheetLR"].append(sheet_lr)
        data["NamaSheetRB"].append(sheet_rb)
        data["NamaFile"].append(namefile)
    
    print("Pengumpulan Nama Sheet Selesai.\r")
    return pd.DataFrame(data)

In [38]:
data = file_info_scraper(folderpath)

Mengumpulkan Nama Sheet...


 48%|████▊     | 351/738 [02:04<04:09,  1.55it/s]

 FinancialStatement-2024-Tahunan-TRUK.xlsx File is not a zip file


100%|██████████| 738/738 [04:11<00:00,  2.94it/s]

Pengumpulan Nama Sheet Selesai.


In [40]:
def xbrl_scraper(jenis_laporan, kolom_sheet):
    
    print("Memulai pengambilan data :")
    
    for sheet_name in tqdm(jenis_laporan):
        data_filtered = data[data[kolom_sheet] == sheet_name].reset_index(drop=True)
        wadah_transit = pd.DataFrame()
        
        for _, row in data_filtered.iterrows():
            try:
                sheet_name = row[kolom_sheet]
                

                if not sheet_name:
                    print(f"Skipping {row['NamaFile']} karena sheet {kolom_sheet} tidak ditemukan.")
                    continue  

                file_path = f"{folderpath}/{row['NamaFile']}"
                file_target = pd.read_excel(file_path, sheet_name=sheet_name, index_col=None)
                file_target = file_target.dropna(how="all").T
                file_target = file_target.drop(file_target.columns[0], axis=1).reset_index(drop=True).drop(3)
                file_target.loc[0, 2] = "Tanggal"
                file_target.columns = file_target.iloc[0].str.lower()
                file_target = file_target[1:].dropna(axis=1, how="all")
                file_target["kode entitas"] = [row["kode entitas"]] * 2
                wadah_transit = pd.concat([wadah_transit, file_target], ignore_index=True)

            except Exception as e:
                print(f"Error pada {row['kode_entitas']}: {e}")

        
        wadah_transit.to_excel(f"{data_transit_path}{sheet_name}.xlsx", index=False)
    
    print("Pengumpulan data selesai. \r")

In [41]:
xbrl_scraper(data["NamaSheetPK"].unique(), "NamaSheetPK")
xbrl_scraper(data["NamaSheetLR"].unique(), "NamaSheetLR")
xbrl_scraper(data["NamaSheetRB"].unique(), "NamaSheetRB")

Memulai pengambilan data :


100%|██████████| 10/10 [02:46<00:00, 16.64s/it]


Pengumpulan data selesai. 
Memulai pengambilan data :


100%|██████████| 19/19 [02:38<00:00,  8.34s/it]


Pengumpulan data selesai. 
Memulai pengambilan data :


100%|██████████| 9/9 [02:57<00:00, 19.75s/it]

Pengumpulan data selesai. 


In [42]:
def gabungkan_data(jenis_laporan):
    gabung_all = pd.concat([
        pd.read_excel(f"{data_transit_path}{sheet}.xlsx") for sheet in jenis_laporan
    ], ignore_index=True)
    
    kolom_awal = ["kode entitas"]
    gabung_all = gabung_all[kolom_awal + [col for col in gabung_all.columns if col not in kolom_awal]].fillna(0)
    return gabung_all

In [43]:
laporan_pk_all = gabungkan_data(data["NamaSheetPK"].unique())
laporan_lr_all = gabungkan_data(data["NamaSheetLR"].unique())
laporan_rb_all = gabungkan_data(data["NamaSheetRB"].unique())

In [44]:
def pemisah_data(df):
    current_q = df.iloc[::2].reset_index(drop=True)
    previous_q = df.iloc[1::2].reset_index(drop=True)
    return current_q, previous_q

In [45]:
PK_currentQ, PK_previousQ = pemisah_data(laporan_pk_all)
LR_currentQ, LR_previousQ = pemisah_data(laporan_lr_all)
RB_currentQ, RB_previousQ = pemisah_data(laporan_rb_all)

In [46]:
def general_information():
    info_entitas = pd.concat([
        pd.read_excel(f"{folderpath}/{file}", sheet_name="1000000").T.dropna(how="all").reset_index(drop=True).drop(2)
        for file in data["NamaFile"]
    ], ignore_index=True)
    
    info_entitas = info_entitas.drop_duplicates().reset_index(drop=True)
    info_entitas.columns = info_entitas.iloc[0].str.lower()
    info_entitas = info_entitas.drop(info_entitas.columns[:2], axis=1).drop(0)
    
    
    return info_entitas

In [47]:
general_info = general_information()

# Stock Scrapper From Google Finance

In [48]:
def stock_latest_googlefinance () :

    datas = data["kode entitas"].unique()

    harga_stock = []

    for ticker in tqdm(datas) :

        url = f'https://www.google.com/finance/quote/{ticker}:IDX?hl=en'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        stocknya = soup.find('div', class_='AHmHk').text
        stocknya = "".join(filter(str.isdigit, stocknya))
        stocknya = int(stocknya) // 100
        
        harga_stock.append(stocknya)
        time.sleep(1)
        
    all_stocks = {
        "kode perusahaan" : datas,
        "penutupan" : harga_stock
    }

    all_stocks = pd.DataFrame(all_stocks)
    
    print("Selesai")
    return all_stocks

In [49]:
latest_stock = stock_latest_googlefinance()

100%|██████████| 737/737 [25:31<00:00,  2.08s/it]

Selesai


# Combine All Datas

In [ ]:
with pd.ExcelWriter(f"{data_hasil_path}Data_Laporan_{tahun}_{kuartal}.xlsx", engine='openpyxl') as writer:
    general_info.to_excel(writer, sheet_name="gen_info", index=False)
    PK_currentQ.to_excel(writer, sheet_name="pk_now", index=False)
    LR_currentQ.to_excel(writer, sheet_name="lr_now", index=False)
    RB_currentQ.to_excel(writer, sheet_name="rb_now", index=False)
    PK_previousQ.to_excel(writer, sheet_name="pk_prev", index=False)
    LR_previousQ.to_excel(writer, sheet_name="lr_prev", index=False)
    RB_previousQ.to_excel(writer, sheet_name="rb_prev", index=False)

In [ ]:
with pd.ExcelWriter(f"{data_hasil_path}Data_Laporan_{tahun}_{kuartal}.xlsx", engine='openpyxl', mode='a') as writer:
    latest_stock.to_excel(writer, sheet_name="stock_info", index=False)